In [6]:
import numpy as np
import modules
from scipy.optimize import minimize

def sensitivity_sweep(v1_name, v1_ratios, v2_name, all_vars):
    # v1: variable being swept, held constant for each optimization
    # v2: variable being optimized

    v1_list = modules.variable_lookup(v1_name)
    v2_list = modules.variable_lookup(v2_name);
    v1_nom = modules.default_values(v1_name);
    num_v1 = len(v1_list);
    num_v2 = len(v2_list);
    num_ratios = len(v1_ratios);
    
    v2_star = np.zeros(shape=(num_v1,num_ratios,num_v2))
    dv2_star_dv1 = np.zeros(shape=(num_v1,num_v2))

    v2_label = ''
    for i in range(len(v2_name)):
        if (i!=0):
            v2_label += ' & '
        v2_label += v2_name[i]
    v1_label = ''
    for i in range(len(v1_name)):
        if (i!=0):
            v1_label += ' & '
        v1_label += v1_name[i]
    desc = 'Finding optimal '+ v2_label + ' while holding '+ v1_label + ' constant.'
    print(desc)
    
    for i in range(num_v1):
        for j in range(num_ratios):
            v1 = v1_nom
            v1[v1_list[i]] = v1_nom[v1_list[i]] * v1_ratios[j]
            res_opt = run_optimization(v2_name,v1_name,v1, all_vars)
            #print(res_opt)
            v2_star[i,j,:] = res_opt['x']
            #print('>>>>>',v2_star[i,j,:])

        idx_nom = ratios.index(1)
        dv2_star = np.divide((v2_star[i,-1,:] - v2_star[i,0,:]), v2_star[i,idx_nom,:])
        dv2_star_dv1[i,:] = dv2_star / (v1_ratios[-1] - v1_ratios[0])    

    # print result in a table format
    print('\n')
    col_width = 15
    print(' '*col_width, end='')
    for i in range(num_v2):
        print(' ',v2_list[i], ' '*(col_width-len(v2_list[i])-2),end='')
    print('')
    for i in range(num_v1):
        print(v1_list[i], ' '*(col_width-len(v1_list[i])),end='|')
        for j in range(num_v2):
            if (dv2_star_dv1[i,j] == 0):
                print('  0', ' '*(col_width-4),end='|')
            else:
                print("{:10.6f}".format(dv2_star_dv1[i,j]), ' '*(col_width-11),end='|')
        print('')
        
    return dv2_star_dv1


def run_optimization(x_name,p_name,p_vals, all_vars):
    # optimizes the design variables x_name, with parameters p_name set to 
    # non-default values p_vals, and other parameters set to default values.
    
    # create optimization variable
    x_list = modules.variable_lookup(x_name)
    x_label = ''
    for i in range(len(x_name)):
        x_label += x_name[i] + '%'
    x = [x_label,x_list]

    x_struct={}
    # create struct to hold optimization variable
    for i in range(len(x_list)):
        x_struct[x_list[i]] = 0 #x[x_list[i]]
    
    # fill default parameters
    str_ = x_name + p_name
    
    default_vars = []
    for i in range(len(all_vars)):
        if all_vars[i] not in str_:
            default_vars.append(all_vars[i])
    #is_default = ~contains(all_vars,[x_name p_name]);
    #default_vars = all_vars[is_default]
    p = modules.default_values(default_vars)

    # fill non-default parameters
    p_label = ''
    if p_name!=[]:
        for i in range(len(p_name)):
            p_label += p_name[i] + '%'
        p_list = modules.variable_lookup(p_name);
        for i in range(len(p_list)):
            p[p_list[i]] = p_vals[p_list[i]]
    
    #desc = 'Finding optimal '+ x_label + ' while holding '+ p_label + ' constant.'
    #print(desc)
    
    # design variables
    x_list_default_values = modules.default_values(x_name)
    x_list_bnds_values = modules.bnds_values(x_name)
    x0 = []
    x0_bnds = []
    for i in range(len(x_list)):
        x0.append(x_list_default_values[x_list[i]])
        x0_bnds.append(x_list_bnds_values[x_list[i]])
        
    # set up optimization problem
    J = modules.obj(x0, x_name, p)
    
    #optimization
    arguments = (x_name, p)
    cons = ({'type': 'ineq', 'fun': modules.ineq_constraint, 'args': arguments},
            {'type': 'eq', 'fun': modules.eq_constraint, 'args': arguments})

    res = minimize(modules.obj, x0, args=arguments, method='SLSQP', bounds=x0_bnds, constraints=cons)

    return res



all_vars = ['x_wec','x_wec_type','x_pen','p_pen','x_env','p_wec','p_fish_salmon']
ratios = [.8, 1, 1.2]
# sensitivity of optimal wec and pen design to environment/location
dx_wec_pen_star_dx_env = sensitivity_sweep(['x_env'], ratios, ['x_wec','x_pen'], all_vars)
#print('\n\ndx_wec_pen_star_dx_env=\n', dx_wec_pen_star_dx_env)
print('\n*************************************************\n\n')

# sensitivity of optimal environment/location to wec and pen design
dx_env_star_dx_wec_pen = sensitivity_sweep(['x_wec','x_pen'], ratios, ['x_env'], all_vars)
#print('\n\ndx_env_star_dx_wec_pen=\n', dx_env_star_dx_wec_pen)
print('\n*************************************************\n\n')

Finding optimal x_wec & x_pen while holding x_env constant.


                 capture_width   pen_diameter    pen_height    
temp            |  0            | -0.315376     | -0.000000     |
O2_in           |  0            |  0.154808     |  0            |
salinity        |  0            |  0            |  0            |
wave_height     |  0            | -0.259162     |  0.000000     |
wave_period     |  0            |  0.563116     |  0.000000     |
U_min           |  0            |  0.144773     |  0            |

*************************************************


Finding optimal x_env while holding x_wec & x_pen constant.


                 temp            O2_in           salinity        wave_height     wave_period     U_min         
capture_width   |  0            |  0            |  0            |  0.000000     | -0.026477     |  0            |
pen_diameter    |  0.000000     |  0            |  0            | -0.000000     | -0.000000     |  0            |
pen_height      | -0.00